<a href="https://colab.research.google.com/github/imabari/pdf2csv/blob/main/%E7%96%BE%E6%82%A3%E5%88%A5%E6%89%8B%E8%A1%93%E5%88%A5%E9%9B%86%E8%A8%88_%E6%96%BD%E8%A8%AD%E9%A1%9E%E5%9E%8B%E5%88%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実行方法

1. 上の「Open in Colab」をクリック（Googleアカウント必要）
2. 設定の年度とURLを入力
3. 上部メニューより「ランタイム」を選択
4. 「すべてのセルを実行」を選択
5. 前処理したデータが自動ダウンロードされます

# 設定

In [1]:
#@title 年度

year =  2020#@param {type:"integer"}

In [2]:
#@title URL

url = "https://www.mhlw.go.jp/content/12404000/000917110.xlsx" #@param {type:"string"}


# プログラム

In [3]:
import pandas as pd

In [4]:
df0 = pd.read_excel(url)

In [5]:
# 列名

df0.columns = [
    "疾患コード",
    "疾患名",
    "件数／在院日数",
    "手術診断群分類番号",
    "大学病院本院群_件数",
    "大学病院本院群_割合",
    "DPC特定病院群_件数",
    "DPC特定病院群_割合",
    "DPC標準病院群_件数",
    "DPC標準病院群_割合",
    "DPC準備病院_件数",
    "DPC準備病院_割合",
    "出来高算定病院_件数",
    "出来高算定病院_割合",
]

In [6]:
# 補完

df0["疾患コード"] = df0["疾患コード"].fillna(method="ffill")
df0["疾患名"] = df0["疾患名"].fillna(method="ffill")
df0["件数／在院日数"] = df0["件数／在院日数"].fillna(method="ffill")

In [7]:
# 縦持ちに変換

df1 = df0.set_index(["疾患コード", "疾患名", "件数／在院日数", "手術診断群分類番号"])
df2 = df1.stack().reset_index()

In [8]:
# 年度追加

df2["年度"] = year

In [9]:
# グループ分け

df2[["種類", "区分"]] = df2["level_4"].str.split("_", expand=True)
grp = df2.groupby("区分")

In [10]:
def col_rename(df0, name):

    df0.rename(columns={0: name}, inplace=True)
    df1 = df0.reindex(columns=["年度", "疾患コード", "疾患名", "件数／在院日数", "手術診断群分類番号", "種類", name]).reset_index(drop=True)

    return df1

## 件数

In [11]:
df3 = col_rename(grp.get_group("件数").copy(), "件数")
df3

,年度,疾患コード,疾患名,件数／在院日数,手術診断群分類番号,種類,件数
0,2019,010010,脳腫瘍,件数,99,大学病院本院群,6814.000000
1,2019,010010,脳腫瘍,件数,99,DPC特定病院群,6916.000000
2,2019,010010,脳腫瘍,件数,99,DPC標準病院群,17410.000000
3,2019,010010,脳腫瘍,件数,99,DPC準備病院,868.000000
4,2019,010010,脳腫瘍,件数,99,出来高算定病院,3491.000000
...,...,...,...,...,...,...,...
13023,2019,180060,その他の新生物,在院日数,97（輸血以外の再掲）,大学病院本院群,7.527273
13024,2019,180060,その他の新生物,在院日数,97（輸血以外の再掲）,DPC特定病院群,5.813819
13025,2019,180060,その他の新生物,在院日数,97（輸血以外の再掲）,DPC標準病院群,5.850109
13026,2019,180060,その他の新生物,在院日数,97（輸血以外の再掲）,DPC準備病院,8.542222


In [12]:
df3.to_csv(f"{year}_count.csv", encoding="utf_8_sig")

## 割合

In [13]:
df4 = col_rename(grp.get_group("割合").copy(), "割合")
df4

,年度,疾患コード,疾患名,件数／在院日数,手術診断群分類番号,種類,割合
0,2019,010010,脳腫瘍,件数,99,大学病院本院群,0.528135
1,2019,010010,脳腫瘍,件数,99,DPC特定病院群,0.638420
2,2019,010010,脳腫瘍,件数,99,DPC標準病院群,0.734320
3,2019,010010,脳腫瘍,件数,99,DPC準備病院,0.775000
4,2019,010010,脳腫瘍,件数,99,出来高算定病院,0.877356
...,...,...,...,...,...,...,...
6509,2019,180060,その他の新生物,件数,97（輸血以外の再掲）,大学病院本院群,0.783329
6510,2019,180060,その他の新生物,件数,97（輸血以外の再掲）,DPC特定病院群,0.832874
6511,2019,180060,その他の新生物,件数,97（輸血以外の再掲）,DPC標準病院群,0.833811
6512,2019,180060,その他の新生物,件数,97（輸血以外の再掲）,DPC準備病院,0.781250


In [14]:
df4.to_csv(f"{year}_ratio.csv", encoding="utf_8_sig")

# ダウンロード

In [15]:
from google.colab import files

In [16]:
files.download(f"{year}_count.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
files.download(f"{year}_ratio.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>